In [ ]:
import gradio as gr
import joblib
import pandas as pd
from datetime import datetime
import re


# List of top 20 Indian cities
TOP_20_CITIES = [
    "Delhi", "Mumbai", "Bangalore", "Hyderabad", "Ahmedabad",
    "Chennai", "Kolkata", "Pune", "Jaipur", "Lucknow",
    "Kanpur", "Nagpur", "Indore", "Bhopal", "Patna",
    "Vadodara", "Ludhiana", "Agra", "Nashik", "Amritsar",
]


# Features expected by the model
FEATURES = [
    'T2M',                # Temperature at 2m (°C)
    'RH2M',               # Relative Humidity at 2m (%)
    'WS2M',               # Wind Speed at 2m (m/s)
    'PRECTOTCORR',        # Precipitation corrected (mm)
    'PS',                 # Surface Pressure (Pa)
    'CLRSKY_SFC_SW_DWN',  # Clear sky surface shortwave downward irradiance (W/m^2)
    'ALLSKY_KT',          # All sky clearness index
    'Month',
    'DayOfYear',
    'Weekday'
]


def load_model(city_name):
    filename = f"{city_name.lower().replace(' ', '_')}_xgb_model.pkl"
    try:
        model = joblib.load(filename)
        return model
    except FileNotFoundError:
        return None


def predict_solar_output(city, T2M, RH2M, WS2M, PRECTOTCORR, PS,
                         CLRSKY_SFC_SW_DWN, ALLSKY_KT, date_str):
    # Load model for the city
    model = load_model(city)
    if model is None:
        return f"❌ Model file for {city} not found. Make sure {city.lower().replace(' ', '_')}_xgb_model.pkl exists."

    # Parse date and extract month, day of year, weekday
    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        return "❌ Invalid date format. Use YYYY-MM-DD."

    Month = date_obj.month
    DayOfYear = date_obj.timetuple().tm_yday
    Weekday = date_obj.weekday()

    # Create feature vector
    input_dict = {
        'T2M': float(T2M),
        'RH2M': float(RH2M),
        'WS2M': float(WS2M),
        'PRECTOTCORR': float(PRECTOTCORR),
        'PS': float(PS),
        'CLRSKY_SFC_SW_DWN': float(CLRSKY_SFC_SW_DWN),
        'ALLSKY_KT': float(ALLSKY_KT),
        'Month': Month,
        'DayOfYear': DayOfYear,
        'Weekday': Weekday
    }

    # Ensure feature order matches training
    X = pd.DataFrame([input_dict], columns=FEATURES)

    # Predict
    try:
        pred = model.predict(X)[0]
    except Exception as e:
        return f"❌ Prediction error: {str(e)}"

    return f"🌞 Predicted Solar Radiation : {pred:.2f} W/m²"


def extract_prediction(output_str):
    """Extract first floating point number from string safely."""
    match = re.search(r"([-+]?\d*\.\d+|\d+)", output_str)
    if match:
        return float(match.group(0))
    else:
        return None


def compare_solar_output(
    city1, city2,
    T2M_1, RH2M_1, WS2M_1, PRECTOTCORR_1, PS_1,
    CLRSKY_SFC_SW_DWN_1, ALLSKY_KT_1, date_str_1,
    T2M_2, RH2M_2, WS2M_2, PRECTOTCORR_2, PS_2,
    CLRSKY_SFC_SW_DWN_2, ALLSKY_KT_2, date_str_2
):
    output1 = predict_solar_output(
        city1, T2M_1, RH2M_1, WS2M_1, PRECTOTCORR_1,
        PS_1, CLRSKY_SFC_SW_DWN_1, ALLSKY_KT_1, date_str_1
    )
    output2 = predict_solar_output(
        city2, T2M_2, RH2M_2, WS2M_2, PRECTOTCORR_2,
        PS_2, CLRSKY_SFC_SW_DWN_2, ALLSKY_KT_2, date_str_2
    )

    pred1 = extract_prediction(output1)
    pred2 = extract_prediction(output2)

    if pred1 is None or pred2 is None:
        compare_text = "Could not compare due to errors in prediction."
    else:
        if pred1 > pred2:
            compare_text = (f"🔶 {city1} is predicted to receive higher solar radiation "
                            f"({pred1:.2f} W/m²) than {city2} ({pred2:.2f} W/m²).")
        elif pred2 > pred1:
            compare_text = (f"🔷 {city2} is predicted to receive higher solar radiation "
                            f"({pred2:.2f} W/m²) than {city1} ({pred1:.2f} W/m²).")
        else:
            compare_text = f"Both cities are predicted to receive equal solar radiation ({pred1:.2f} W/m²)."

    result = (
        f"{city1}: {output1}\n"
        f"{city2}: {output2}\n\n"
        f"{compare_text}"
    )
    return result


def build_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🔆 Solar Radiation Prediction App for Indian Cities")

        with gr.Tab("Single City Prediction"):
            gr.Markdown(
                "## Single City Solar Radiation Prediction\n"
                "Select a city, provide weather conditions & date, and get the predicted solar radiation."
            )
            with gr.Row():
                city_sel = gr.Dropdown(TOP_20_CITIES, label="Select City", value=TOP_20_CITIES[0])
            with gr.Row():
                T2M = gr.Number(label="Temperature at 2m (°C)", value=30.0)
                RH2M = gr.Number(label="Relative Humidity at 2m (%)", value=50.0)
                WS2M = gr.Number(label="Wind Speed at 2m (m/s)", value=2.0)
                PRECTOTCORR = gr.Number(label="Precipitation (mm)", value=0.0)
                PS = gr.Number(label="Surface Pressure (Pa)", value=101325)
                CLRSKY_SFC_SW_DWN = gr.Number(label="Clear Sky Surface SW Downward (W/m²)", value=200.0)
                ALLSKY_KT = gr.Number(label="All Sky Clearness Index", value=0.6)
            date_input = gr.Textbox(label="Date (YYYY-MM-DD)", value=datetime.today().strftime("%Y-%m-%d"))

            predict_btn = gr.Button("Predict Solar Radiation")
            output = gr.Textbox(label="Prediction Output")

            predict_btn.click(
                fn=predict_solar_output,
                inputs=[city_sel, T2M, RH2M, WS2M, PRECTOTCORR,
                        PS, CLRSKY_SFC_SW_DWN, ALLSKY_KT, date_input],
                outputs=output
            )

        with gr.Tab("Compare Two Cities"):
            gr.Markdown("## Compare Solar Radiation Between Two Indian Cities")

            with gr.Row():
                city1_sel = gr.Dropdown(TOP_20_CITIES, label="City 1", value=TOP_20_CITIES[0])
                city2_sel = gr.Dropdown(TOP_20_CITIES, label="City 2", value=TOP_20_CITIES[1])

            gr.Markdown("### Inputs for City 1")
            with gr.Row():
                T2M_1 = gr.Number(label="Temperature at 2m (°C)", value=30.0)
                RH2M_1 = gr.Number(label="Relative Humidity at 2m (%)", value=50.0)
                WS2M_1 = gr.Number(label="Wind Speed at 2m (m/s)", value=2.0)
                PRECTOTCORR_1 = gr.Number(label="Precipitation (mm)", value=0.0)
                PS_1 = gr.Number(label="Surface Pressure (Pa)", value=101325)
                CLRSKY_SFC_SW_DWN_1 = gr.Number(label="Clear Sky SW Downward (W/m²)", value=200.0)
                ALLSKY_KT_1 = gr.Number(label="All Sky Clearness Index", value=0.6)
            date_input_1 = gr.Textbox(label="Date (YYYY-MM-DD)", value=datetime.today().strftime("%Y-%m-%d"))

            gr.Markdown("### Inputs for City 2")
            with gr.Row():
                T2M_2 = gr.Number(label="Temperature at 2m (°C)", value=30.0)
                RH2M_2 = gr.Number(label="Relative Humidity at 2m (%)", value=50.0)
                WS2M_2 = gr.Number(label="Wind Speed at 2m (m/s)", value=2.0)
                PRECTOTCORR_2 = gr.Number(label="Precipitation (mm)", value=0.0)
                PS_2 = gr.Number(label="Surface Pressure (Pa)", value=101325)
                CLRSKY_SFC_SW_DWN_2 = gr.Number(label="Clear Sky SW Downward (W/m²)", value=200.0)
                ALLSKY_KT_2 = gr.Number(label="All Sky Clearness Index", value=0.6)
            date_input_2 = gr.Textbox(label="Date (YYYY-MM-DD)", value=datetime.today().strftime("%Y-%m-%d"))

            compare_btn = gr.Button("Compare Solar Radiation")
            output_compare = gr.Textbox(label="Comparison Output")

            compare_btn.click(
                fn=compare_solar_output,
                inputs=[
                    city1_sel, city2_sel,
                    T2M_1, RH2M_1, WS2M_1, PRECTOTCORR_1, PS_1,
                    CLRSKY_SFC_SW_DWN_1, ALLSKY_KT_1, date_input_1,
                    T2M_2, RH2M_2, WS2M_2, PRECTOTCORR_2, PS_2,
                    CLRSKY_SFC_SW_DWN_2, ALLSKY_KT_2, date_input_2
                ],
                outputs=output_compare
            )

    return demo


if __name__ == "__main__":
    app = build_interface()
    app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e0f32d91e3fbde1e84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
